In [1]:
%pwd

'e:\\Vscode\\MLOPS-1\\Chest_Disease_Classification\\research'

In [4]:
import os
os.chdir("../")


In [5]:
%pwd

'e:\\Vscode\\MLOPS-1\\Chest_Disease_Classification'

In [27]:
from dataclasses import dataclass

from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [22]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self)->ModelTrainerConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chest-CT-Scan-data")
        create_directories([Path(training.root_dir)])

        training_config=ModelTrainerConfig(
            root_dir=Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    




In [24]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [31]:
from scipy import interpolate


class Training:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20)
        
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
            )
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator
        
        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
            )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [33]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-15 10:08:25,348: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-03-15 10:08:25,395: INFO: common: yaml file:params.yaml loaded successfully]
[2024-03-15 10:08:25,398: INFO: common: directory:artifacts created successfully]


[2024-03-15 10:08:25,401: INFO: common: directory:artifacts\training created successfully]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/8
17/17 [==============================] - 98s 6s/step - loss: 14.3416 - accuracy: 0.5367 - val_loss: 5.1378 - val_accuracy: 0.6094
Epoch 2/8
17/17 [==============================] - 98s 6s/step - loss: 14.5180 - accuracy: 0.5521 - val_loss: 0.0221 - val_accuracy: 0.9844
Epoch 3/8
17/17 [==============================] - 98s 6s/step - loss: 13.3549 - accuracy: 0.5560 - val_loss: 0.0627 - val_accuracy: 0.9844
Epoch 4/8
17/17 [==============================] - 119s 7s/step - loss: 3.1924 - accuracy: 0.7954 - val_loss: 0.0160 - val_accuracy: 1.0000
Epoch 5/8
17/17 [==============================] - 148s 9s/step - loss: 8.0010 - accuracy: 0.6486 - val_loss: 0.1220 - val_accuracy: 0.9844
Epoch 6/8
17/17 [==============================] - 133s 8s/step - loss: 1.9084 - accuracy: 0.8224 - val_loss: 0.0059 - val_accur